### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6817,2023-11-27,Brasil Nbb,19:00,Flamengo,Bauru,1.23,3.86,149.5,1.82,1.88,-10.5,2.04,1.69,rH56qqf3,0.813008,0.259067,0.549451,0.531915,0.072075,0.0,0.0,NaN,NaN,92.434,9.529721,0.103098,141.748,68.830176,0.485581,0.000,0.000,94.76,264.00,0.0,0.0,0.0,0.0,0.730723,0.022933,0.132701,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6818,2023-11-27,Brasil Nbb,19:30,São José,R10 Score Vasco da Gama,1.75,1.98,148.5,1.87,1.79,-2.5,2.02,1.71,4Q62pPvc,0.571429,0.505051,0.534759,0.558659,0.076479,0.6,0.0,NaN,NaN,195.284,88.781711,0.454629,NaN,NaN,NaN,186.292,0.000,115.20,0.00,0.0,0.0,0.0,0.0,0.087204,0.030912,0.117535,1.40,0.280,2.678571,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6819,2023-11-27,Eua Nba,21:00,Detroit Pistons,Washington Wizards,1.66,2.43,232.5,1.88,2.05,-3.5,2.00,1.94,jkJWKAvr,0.602410,0.411523,0.531915,0.487805,0.013932,0.8,0.4,NaN,NaN,384.954,43.986249,0.114264,507.636,267.336970,0.526631,322.994,488.762,333.60,952.32,0.0,1.0,0.0,0.0,0.266246,0.061175,0.021536,-2.95,-0.590,-1.118644,0.259431,0.1,-0.159431,-2.56,-0.512,-2.792969,0.266865,0.1,-0.166865
6820,2023-11-27,Eua Nba,21:00,Indiana Pacers,Portland Trail Blazers,1.16,6.44,238.5,1.80,2.08,-11.5,2.04,1.88,UuIzKUgl,0.862069,0.155280,0.555556,0.480769,0.017348,1.0,0.6,NaN,NaN,330.418,92.706215,0.280573,536.766,287.371345,0.535375,215.000,569.006,178.16,831.30,0.0,1.0,0.0,0.0,0.982506,0.102057,0.057723,1.53,0.306,0.522876,0.600905,0.7,0.099095,-5.00,-1.000,-5.440000,0.196716,0.1,-0.096716
6821,2023-11-27,Eua Nba,21:00,Philadelphia 76ers,Los Angeles Lakers,1.50,2.87,229.5,1.81,2.07,-5.5,1.99,1.95,O2HvJl9f,0.666667,0.348432,0.552486,0.483092,0.015099,0.8,0.6,NaN,NaN,217.810,40.155107,0.184358,182.474,39.107436,0.214318,196.036,190.262,158.27,136.96,0.0,0.0,0.0,0.0,0.443358,0.094767,0.014357,-1.44,-0.288,-1.736111,0.611839,0.6,-0.011839,-1.52,-0.304,-6.151316,0.460300,0.3,-0.160300
6822,2023-11-27,Eua Nba,23:00,Utah Jazz,New Orleans Pelicans,2.51,1.62,228.5,1.88,2.05,3.5,2.00,1.94,x6LrI8O0,0.398406,0.617284,0.531915,0.487805,0.015690,0.6,0.4,NaN,NaN,296.210,92.566412,0.312503,278.626,65.804675,0.236176,264.498,298.920,206.85,197.00,0.0,0.0,0.0,0.0,0.304758,0.061175,0.021536,-1.58,-0.316,-4.778481,0.447850,0.5,0.052150,0.27,0.054,11.481481,0.442634,0.5,0.057366
6823,2023-11-27,Alemanha Bbl,16:00,Syntainics MBC,Wurzburg,1.78,2.04,163.5,1.81,1.97,-2.5,2.02,1.74,vZZu2b2E,0.561798,0.490196,0.552486,0.507614,0.051994,0.0,0.0,NaN,NaN,420.052,409.062932,0.973839,288.628,280.084200,0.970399,0.000,0.000,174.60,176.61,0.0,0.0,0.0,0.0,0.096255,0.059861,0.105314,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6824,2023-11-27,Argentina Liga A,20:00,Obras,La Union,1.28,3.44,165.5,1.85,1.85,-9.5,2.10,1.65,EZ86Cooa,0.781250,0.290698,0.540541,0.540541,0.071948,0.0,0.0,NaN,NaN,180.056,65.661530,0.364673,NaN,NaN,NaN,0.000,0.000,214.24,143.25,0.0,0.0,0.0,0.0,0.647182,0.000000,0.169706,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6825,2023-11-27,Argentina Liga A,21:10,Olimpico,Comunicaciones Mercedes,1.09,6.47,163.5,1.84,1.86,-14.5,1.98,1.74,dKk7gCNM,0.917431,0.154560,0.543478,0.537634,0.071991,0.0,0.0,NaN,NaN,129.988,31.197565,0.240003,NaN,NaN,NaN,0.000,0.000,93.09,286.35,0.0,0.0,0.0,0.0,1.006411,0.007644,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6826,2023-11-27,Argentina Liga A,22:00,Qui

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,21:00,Eua Nba,Detroit Pistons,Washington Wizards,232.5,1.88,1.0000,Over
3,21:00,Eua Nba,Indiana Pacers,Portland Trail Blazers,238.5,1.80,0.9999,Over
4,21:00,Eua Nba,Philadelphia 76ers,Los Angeles Lakers,229.5,1.81,1.0000,Over
5,23:00,Eua Nba,Utah Jazz,New Orleans Pelicans,228.5,1.88,1.0000,Over
